<a href="https://colab.research.google.com/github/jiheddachraoui/occupancy_grid_generator/blob/main/Evaluate_loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PREPARATION

In [ ]:
import os
import sys
import csv
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time
from PIL import Image
import tensorflow as tf
from sklearn.linear_model import RANSACRegressor
from scipy import stats
from google.colab import drive
drive.mount('/content/drive',force_remount=True)


Mounted at /content/drive


In [ ]:
datadir = '/content/drive/MyDrive/s_data/'

gt_path=datadir+'ground truth'
res_path=datadir+'output'

scenario='scenario_test2_gt'


for item in os.listdir(os.path.join(res_path,scenario)):
    # Construct the full path to the item
    item_path = os.path.join(os.path.join(res_path,scenario), item)
    # Check if the item is a directory
    if os.path.isdir(item_path) and item != '.ipynb_checkpoints':
        # Print the path of the subdirectory
        print(item_path)



/content/drive/MyDrive/s_data/output/scenario_test2_gt/SG fusion
/content/drive/MyDrive/s_data/output/scenario_test2_gt/OG fusion


In [ ]:

vehicles=[]
for item in os.listdir(os.path.join(gt_path,scenario)):
  vehicles.append(item)
vehicle=vehicles[1]

folder_path = os.path.join(gt_path,scenario,vehicle)
out_path= os.path.join(res_path,scenario)
print(folder_path)
print(out_path)

/content/drive/MyDrive/s_data/ground truth/scenario_test2_gt/adversary vehicle.tesla.model3
/content/drive/MyDrive/s_data/output/scenario_test2_gt


In [ ]:
vehicles=[]
for item in os.listdir(os.path.join(gt_path,scenario)):
    # Construct the full path to the item
    print(item)
    if os.path.isdir(os.path.join(gt_path,scenario,item)) and item != '.ipynb_checkpoints':
        folder_path = os.path.join(os.path.join(gt_path,scenario), item)

        og=os.path.join(out_path, "OG fusion/fusion_image")
        sg=os.path.join(out_path, "SG fusion/fusion_image")

        ogt_path=os.path.join(folder_path, "grids/bird_ogm_gt")
        sgt_path=os.path.join(folder_path, "grids/bird_eye_image_semantic_grid")

        num_files = len([f for f in os.listdir(og) if os.path.isfile(os.path.join(og, f))])
        print('Number of files in {} :'.format("OG fusion/fusion_image"), num_files)
        vehicles.append((og,ogt_path,sg,sgt_path,num_files))

hero vehicle.lincoln.mkz_2017
Number of files in OG fusion/fusion_image : 406
adversary vehicle.tesla.model3
Number of files in OG fusion/fusion_image : 406


In [ ]:
vehicles

[('/content/drive/MyDrive/s_data/output/scenario_test2_gt/OG fusion/fusion_image',
  '/content/drive/MyDrive/s_data/ground truth/scenario_test2_gt/hero vehicle.lincoln.mkz_2017/grids/bird_ogm_gt',
  '/content/drive/MyDrive/s_data/output/scenario_test2_gt/SG fusion/fusion_image',
  '/content/drive/MyDrive/s_data/ground truth/scenario_test2_gt/hero vehicle.lincoln.mkz_2017/grids/bird_eye_image_semantic_grid',
  406),
 ('/content/drive/MyDrive/s_data/output/scenario_test2_gt/OG fusion/fusion_image',
  '/content/drive/MyDrive/s_data/ground truth/scenario_test2_gt/adversary vehicle.tesla.model3/grids/bird_ogm_gt',
  '/content/drive/MyDrive/s_data/output/scenario_test2_gt/SG fusion/fusion_image',
  '/content/drive/MyDrive/s_data/ground truth/scenario_test2_gt/adversary vehicle.tesla.model3/grids/bird_eye_image_semantic_grid',
  406)]

In [ ]:
def generate_semantic_mask(ground_truth, window_size):
    # Convert ground truth to binary occupancy grid
    gt_road = np.all(ground_truth == [128, 64, 128], axis=2)

    # Create a boolean mask by expanding the "free" pixels within a window size
    mask = np.zeros_like( gt_road, dtype=bool)
    for i in range(window_size // 2,  gt_road.shape[0] - window_size // 2):
        for j in range(window_size // 2,  gt_road.shape[1] - window_size // 2):
            if  gt_road[i, j]:
                mask[i - window_size // 2 : i + window_size // 2 + 1, j - window_size // 2 : j + window_size // 2 + 1] = 1

    return mask

In [ ]:
def generate_mask(ground_truth, window_size):
    # Convert ground truth to binary occupancy grid
    gt_free = np.all(ground_truth == [255, 255, 255], axis=2)

    # Create a boolean mask by expanding the "free" pixels within a window size
    mask = np.zeros_like(gt_free, dtype=bool)
    for i in range(window_size // 2, gt_free.shape[0] - window_size // 2):
        for j in range(window_size // 2, gt_free.shape[1] - window_size // 2):
            if gt_free[i, j]:
                mask[i - window_size // 2 : i + window_size // 2 + 1, j - window_size // 2 : j + window_size // 2 + 1] = 1

    return mask

In [ ]:
def predict(dgm,dt=0.5):

  pred_map = np.zeros((dgm.shape[0],dgm.shape[1],3))
  # The unknown cells: gray
  pred_map[dgm==0.5] = (123,123,123)
  # The occupied cells: black
  pred_map[dgm==1] = (0,0,0)
  # The free cells: white
  pred_map[dgm==0] = (255,255,255)

  return pred_map.astype(np.uint8)

In [ ]:
def load_matrix(folder_path,timestamp):
  ts = str(timestamp)
  for file_name in os.listdir(folder_path):
          if os.path.splitext(file_name)[0] == ts:

              file_path = os.path.join(folder_path,file_name)
              #print(file_path)
              break

  m=np.load(file_path)
  return m

In [ ]:
def evaluate_ogm_rgb(gt_image, pred_image):


    gt_occupied = np.all(gt_image == [0, 0, 0], axis=2)
    pred_occupied = np.all(pred_image == [0, 0, 0], axis=2)

    # Calculate metrics
    accuracy = np.sum(gt_occupied == pred_occupied) / gt_occupied.size

    # Calculate precision and recall
    true_positives = np.sum(np.logical_and(gt_occupied, pred_occupied))
    false_positives = np.sum(np.logical_and(np.logical_not(gt_occupied), pred_occupied))
    false_negatives = np.sum(np.logical_and(gt_occupied, np.logical_not(pred_occupied)))
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)

    # Compute F1 score
    f1_score = 2 * (precision * recall) / (precision + recall)

    # Return evaluation metrics
    return accuracy, precision, recall, f1_score

In [ ]:
def evaluate_semantic_ogm_rgb(gt_image, pred_image):
    # Extract the unique classes present in the ground truth and predicted images
    gt_classes = np.unique(gt_image.reshape(-1, 3), axis=0)
    pred_classes = np.unique(pred_image.reshape(-1, 3), axis=0)

    # Combine the unique classes from both images
    classes = np.unique(np.concatenate((gt_classes, pred_classes), axis=0), axis=0)

    # Initialize evaluation metrics
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    true_negatives = 0

    # Iterate over each class
    for class_val in classes:
        # Create boolean masks for the current class in the ground truth and predicted images
        gt_mask = np.all(gt_image == class_val, axis=2)
        pred_mask = np.all(pred_image == class_val, axis=2)

        # Count true positives, false positives, and false negatives
        true_positives += np.sum(np.logical_and(gt_mask, pred_mask))
        true_negatives += np.sum(np.logical_and(np.logical_not(gt_mask), np.logical_not(pred_mask)))
        false_positives += np.sum(np.logical_and(np.logical_not(gt_mask), pred_mask))
        false_negatives += np.sum(np.logical_and(gt_mask, np.logical_not(pred_mask)))

    # Calculate precision, recall, and F1 score
    accuracy = (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_negatives)

    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    f1_score = 2 * (precision * recall) / (precision + recall)

    # Return evaluation metrics
    return accuracy,precision, recall, f1_score

In [ ]:
og_metrics=[]
sg_metrics=[]


'''for item in vehicles:
  print(item)'''
item=vehicles[1]
half_point, remainder = divmod(item[-1], 2)
for  i in range(half_point,354):
    #print(i)
    og=item[0]
    ogt_path=item[1]
    sg=item[2]
    sgt_path=item[3]
    ogm_image=np.array(Image.open(os.path.join(og, "{}.png".format(i))))
    sgm_image=np.array(Image.open(os.path.join(sg, "{}.png".format(i))))
    gt=load_matrix(os.path.join(ogt_path),i)
    sgt_img=load_matrix(os.path.join(sgt_path),i)
    gt_image=predict(gt)
    mask=generate_mask(ogm_image, 100)

    gt_image_m = np.copy(gt_image)
    gt_image_m[~mask] = (123,123,123)

    ogm_image_m = np.copy(ogm_image)
    ogm_image_m[~mask] = (123,123,123)

    mask1=generate_semantic_mask(sgm_image,110)

    sgt_img_m = np.copy(sgt_img)
    sgt_img_m[~mask1] = (1,1,1)

    sgm_image_m = np.copy(sgm_image/ 255.0)
    sgm_image_m[~mask1] = (1,1,1)
    # Evaluate the occupancy grid algorithm
    # Evaluate OG metrics for the current frame
    accuracy_og, precision_og, recall_og, f1_score_og = evaluate_ogm_rgb(gt_image_m, ogm_image_m)
    # Append the metrics to the OG metrics list
    og_metrics.append([i, accuracy_og, precision_og, recall_og, f1_score_og])

    # Evaluate semantic OG metrics for the current frame
    accuracy_sg, precision_sg, recall_sg, f1_score_sg = evaluate_semantic_ogm_rgb(sgt_img_m, sgm_image_m)
    # Append the metrics to the SG metrics list
    sg_metrics.append([i, accuracy_sg, precision_sg, recall_sg, f1_score_sg])

# Save OG metrics to CSV
with open(out_path+'/og_metrics1.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Frame ID', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])
    writer.writerows(og_metrics)

# Save semantic OG metrics to CSV
with open(out_path+'/sg_metrics1.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Frame ID', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])
    writer.writerows(sg_metrics)
